# Simulator sketch up

## Simulation type
1. Link-based
2. Vehicle based

## Data source:
1. Historical data as stream
2. Live data as stream

## Objective 1: Imitate real-time trajectories
1. Set up route and stops (We dont care about the best practice right now)

# 1. Setup

In [14]:
import os
import sys
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.stats as ss

from IPython.display import display, clear_output

import buskit as bk
from buskit import busdata as bdata
from buskit import dashboard as bdash
from buskit import simulator as bsim

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/Yuwen/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## 1.1 Global Functions

In [31]:
# file reader
def csvRows(filename, dir_ref):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            if row['DirectionRef'] == str(dir_ref):
                yield row

## 1.2 Classes

## 1.3 Global Parameters

In [22]:
# determine data source
beta = False
time_coef = 100000 # simulation time is __ times faster than the reality
avg_door_t = 5 # assume opening and closing the door take 5 seconds in total
avg_board_t = 3 # assume each boarding takes 3 sec
avg_alight_t = 2 # assume each alight takes 2 sec

if beta:
    # artificial data   ### make this part automatized with given number of stop
    stop_ref = np.array([1, 2, 3, 4, 5, 6, 7])
    stop_pos = np.array([0, 100, 200, 300, 400, 500, 600])
    stop_name = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G'])
else:
    # historical data
    stop_ref, stop_pos, stop_name = bsim.read_data("MTA_data/B15-180625-235941-44650-Mon.csv", 1)

# speed and travel time data are currently artificial
link_vel = 1.5 * np.random.randn(len(stop_pos)) + 7 # make sure the unit is m/sec
#dwell_t = 7 * np.random.randn(len(stop_pos)) + 20 # make sure the unit is sec

# pax distribution
stop_pos_next = np.append(stop_pos, stop_pos[-1])[1:]

pos_mu = stop_pos.mean()
pos_std = stop_pos.std()
pax_norm = ss.norm(loc=pos_mu, scale=pos_std)
pax_perc = np.array([pax_norm.cdf(stop_pos_next[i]) - pax_norm.cdf(stop_pos[i]) for i in range(len(stop_pos))]) ### a temporary measure ###

pax_hr_route = 5000
pax_hr_stop = pax_hr_route * pax_perc
pax_at_stop = np.zeros(len(stop_pos))

# 2. Simulation Environment
## 2.1 Route Setup

In [23]:
# Bus class
class Bus(object):
    
    capacity = 60
    seat = 40
    
    def __init__(self, ref, pos=0):
        self.ref = ref # vehicle reference
        self.pos = pos # vehicle location (1-D)
        
        ############### fix this: stop_pos, link, vel, next_stop should not require pre-specify, how? ##############
        
        self.link = np.sum(self.pos >= stop_pos) - 1 # link index starts from 0  ### unified with the formula in Stop Class
        self.vel = link_vel[self.link] # speed at current link
        self.next_stop = stop_pos[self.link + 1] # position of next stop
        self.dwell_t = 0
        self.pax = 0
        self.clock = 0
        self.operate = True
        self.atstop = False
        
        self.log_pos = [self.pos]
        self.log_vel = [self.vel]
        self.log_pax = [0]
        self.log_dwell = [0]

    def terminal(self):
        print("The bus has reached the terminal")
        self.operate = False
        self.vel = 0
        self.pax = 0
        
    def stop(self):
        print("Bus %s is making a stop at %s (position %i)"%(self.ref, stop_name[self.link + 1], self.next_stop))
        self.atstop = True
        self.pax_to_board = pax_at_stop[self.link + 1] # check how many pax at stop
        self.board_t = self.pax * avg_board_t
        self.alight_t = 0 * avg_alight_t  #### TO DEVELOP
        self.dwell_t = avg_door_t + self.alight_t + self.board_t # supposed to dwell for this long
        self.clock += 1

#         self.vel = 0
#         self.pos += self.vel
        self.record()

    def move(self):
        pax_at_stop[self.link + 1] = 0 # clear all pax at stop
        self.log_dwell.append(self.dwell_t)
        # move on!
        self.atstop = False
        self.dwell_t = 0
        self.clock = 0
        self.link += 1
        self.pax = 0 # update pax onboard ###################
        self.record()
        self.vel = link_vel[self.link] # new link speed
        self.next_stop = stop_pos[self.link + 1] # new next stop

    def record(self):
        self.log_pos.append(self.pos)
        self.log_pax.append(self.pax)
        
    def proceed(self):
        if self.operate:
            if self.pos + self.vel >= stop_pos[-1]:
                self.terminal()
            elif self.pos + self.vel >= self.next_stop:  ### this judgement restricts from recording vel as 0 at stop, change to sth else
                self.stop()
                if self.clock >= self.dwell_t:
                    self.move()
            else:
                print("Current position of bus %s: %i"%(self.ref, self.pos))
                self.pos += self.vel
                self.record()
        else:
            print("Bus %s is not operating."%(self.ref))

In [32]:
data = csvRows('MTA_data/B15-180625-235941-44650-Mon.csv', 1)

In [33]:
bus1 = next(data)

In [34]:
bus1

OrderedDict([('', '2'),
             ('Bearing', '84.67599'),
             ('BlockRef', 'MTA NYCT_EN_B8-Weekday-SDon_E_EN_75960_B25-228'),
             ('CallDistanceAlongRoute', '16606.85'),
             ('DataFrameRef', '2018-06-25'),
             ('DatedVehicleJourneyRef',
              'MTA NYCT_EN_B8-Weekday-SDon-138600_B25_228'),
             ('DestinationName', 'BED-STUY WOODHULL HOSP'),
             ('DestinationRef', 'MTA_901017'),
             ('DirectionRef', '1'),
             ('DistanceFromCall', '22.44'),
             ('ExpectedArrivalTime', '2018-06-25T23:59:42.106-04:00'),
             ('ExpectedDepartureTime', '2018-06-25T23:59:42.106-04:00'),
             ('JourneyPatternRef', 'MTA_B150238'),
             ('Latitude', '40.668624'),
             ('LineRef', 'MTA NYCT_B15'),
             ('Longitude', '-73.922606'),
             ('Monitored', 'True'),
             ('OperatorRef', 'MTA NYCT'),
             ('OriginAimedDepartureTime', ''),
             ('OriginRef', 'MTA

In [35]:
bus1['VehDistAlongRoute']

'16584.41'

In [36]:
# operate as set
# live_bus = set()
# live_bus.add(Bus(bus1['VehicleRef'], bus1['VehDistAlongRoute']))

# operate as list
live_bus = []
live_bus.append(Bus(bus1['VehicleRef'], float(bus1['VehDistAlongRoute'])))

In [38]:
live_bus[0].pos

16584.41

In [66]:
live_bus

{<__main__.Bus at 0x114d73b70>}